<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-23 15:02:39
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.99 L
Current:  1.28 C
-------------------
Today PnL: 73.90 K (0.58%)
Current PnL: -16.88 L (-11.97%)
CY Booked + Current PnL: -12.59 L (-8.93%)
-------------------
Total profit:  3.54 L
Total loss:  -20.43 L
-------------------
Total Booked + Current PnL: 14.39 L (12.62%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.35%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 78.60 L (61.37%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.40 C (109.12%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,1.84,H-LC,86.17,100811.0,17008.0,6986.0,2.32,20.29,6.93,28.63,159.0,2.43,0.82,34.71,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,3.47,X-LC,80.85,153625.0,12816.0,12705.0,-1.39,9.10,8.27,18.13,157.0,1.01,1.25,21.30,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99144.0,20782.0,13246.0,1.82,26.52,13.36,43.43,39.0,1.57,0.81,32.61,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-18.22,X-LC,25.53,90600.0,15478.0,16679.0,0.85,20.60,18.41,42.81,100.0,0.93,0.74,13.62,X40,BTT,CHEMICALS
69,SBIN,760.30,863.00,-14.39,M-LC,30.32,209615.0,8136.0,19075.0,0.73,4.04,9.10,13.51,71.0,0.43,1.70,17.14,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,500.38,672.85,-13.51,H-LC,1.06,148840.0,-3776.0,56381.0,3.93,-2.47,37.88,34.47,12.0,-0.07,1.21,12.12,AR,ATH,FMCG
45,JPPOWER,18.73,26.20,-43.26,L-SC,14.89,115337.0,-26505.0,83077.0,3.61,-18.69,72.03,39.88,136.0,-0.32,0.94,16.88,XY24,NTT,POWER
75,STARHEALTH,551.56,761.00,2.95,M-SC,93.62,168214.0,-32002.0,108027.0,3.47,-15.98,64.22,37.97,110.0,-0.30,1.37,35.06,XY24,NTT,INSURANCE
37,ICICIPRULI,600.83,791.05,-10.45,H-MC,64.89,144663.0,8875.0,34112.0,3.28,6.54,23.58,31.66,184.0,0.26,1.18,19.29,X40,ATH,INSURANCE
5,ALKYLAMINE,2347.98,4567.52,-15.28,X-SC,26.60,78905.0,-22058.0,117497.0,3.21,-21.85,148.91,94.53,75.0,-0.19,0.64,16.98,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,96.81,64800.0,-48749.0,88802.0,-3.05,-42.93,137.04,35.27,182.0,-0.55,0.53,58.60,XR,NTT,HOTELS
87,VAIBHAVGBL,350.21,521.0,57.58,X-SC,63.83,105202.0,-46789.0,120909.0,-2.49,-30.78,114.93,48.77,45.0,-0.39,0.85,20.84,XR,NTT,JEWELLERY
73,SIS,477.00,528.0,2087.95,M-SC,52.13,59228.0,-21862.0,30532.0,-1.98,-26.96,51.55,10.69,162.0,-0.72,0.48,20.03,X40N,NTT,MISC
13,BERGEPAINT,558.54,680.0,-17.95,H-MC,8.51,176157.0,-900.0,39406.0,-1.89,-0.51,22.37,21.75,191.0,-0.02,1.43,26.01,XY24,NTT,PAINTS
62,RELIANCE,1291.83,1526.0,3.47,X-LC,80.85,153625.0,12816.0,12705.0,-1.39,9.10,8.27,18.13,157.0,1.01,1.25,21.30,XY25,NTT,REFINERIES


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,AWL,342.88,485.00,-69.50,H-MC,5.32,165322.0,-52407.0,142656.0,1.11,-24.07,86.29,41.45,1.0,-0.37,1.34,10.43,XY24,NTT,FMCG
38,INDIAMART,2327.09,4911.36,-51.84,H-SC,50.00,125345.0,2009.0,134959.0,-1.16,1.63,107.67,111.05,2.0,0.01,1.02,22.82,AR,ATH,MISC
81,TCS,3888.13,4998.00,-15.84,X-LC,48.94,211014.0,-22274.0,88858.0,1.15,-9.55,42.11,28.55,3.0,-0.25,1.71,8.83,X40,BTT,IT
41,INFY,1432.81,2275.00,-12.78,X-LC,67.02,217646.0,18485.0,98572.0,1.18,9.28,45.29,58.78,4.0,0.19,1.77,13.63,X40,BTT,IT
80,TATAMOTORS,771.32,1065.00,-52.64,M-LC,75.53,210608.0,-15389.0,101429.0,0.19,-6.81,48.16,38.07,5.0,-0.15,1.71,23.98,XY24,NTT,AUTO


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,2.04,H-SC,98.94,117942.0,20296.0,27374.0,-0.22,20.78,23.21,48.82,185.0,0.74,0.96,31.02,X40,ATH,PHARMA
68,SBILIFE,1496.49,1924.99,1.84,H-LC,86.17,100811.0,17008.0,6986.0,2.32,20.29,6.93,28.63,159.0,2.43,0.82,34.71,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-18.22,X-LC,25.53,90600.0,15478.0,16679.0,0.85,20.60,18.41,42.81,100.0,0.93,0.74,13.62,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99144.0,20782.0,13246.0,1.82,26.52,13.36,43.43,39.0,1.57,0.81,32.61,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,45.79,M-SC,84.04,150974.0,13396.0,81013.0,1.39,9.74,53.66,68.62,60.0,0.17,1.23,30.88,X40N,NTT,TEXTILES
57,POLYCAB,5005.95,7541.35,5.53,H-MC,59.57,100113.0,15012.0,28092.0,-0.99,17.64,28.06,50.65,44.0,0.53,0.81,25.85,X40N,ATH,CABLES
55,PFIZER,4245.50,6318.30,2.04,H-SC,98.94,117942.0,20296.0,27374.0,-0.22,20.78,23.21,48.82,185.0,0.74,0.96,31.02,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,0.94,H-SC,61.70,148903.0,11043.0,26713.0,0.38,8.01,17.94,27.39,87.0,0.41,1.21,29.06,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99144.0,20782.0,13246.0,1.82,26.52,13.36,43.43,39.0,1.57,0.81,32.61,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-3.11,H-SC,10.64,108870.0,6866.0,38159.0,0.14,6.73,35.05,44.14,31.0,0.18,0.88,24.07,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,31.91,133203.0,4051.0,83012.0,0.52,3.14,62.32,67.42,51.0,0.05,1.08,29.55,XR,NTT,BANKS
93,WIPRO,243.46,420.00,-10.62,M-LC,45.74,153549.0,2604.0,106855.0,0.70,1.72,69.59,72.51,28.0,0.02,1.25,15.61,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-51.84,H-SC,50.00,125345.0,2009.0,134959.0,-1.16,1.63,107.67,111.05,2.0,0.01,1.02,22.82,AR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,78.72,150896.0,-17.0,60464.0,0.82,-0.01,40.07,40.06,22.0,-0.00,1.23,23.42,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-55.73,L-SC,96.81,64800.0,-48749.0,88802.0,-3.05,-42.93,137.04,35.27,182.0,-0.55,0.53,58.60,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-23.67,H-SC,57.45,86820.0,-11160.0,33026.0,0.23,-11.39,38.04,22.32,133.0,-0.34,0.71,25.92,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-332.06,H-SC,70.21,98592.0,-34694.0,119286.0,-1.31,-26.03,120.99,63.47,135.0,-0.29,0.80,65.26,XR,NTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99144.0,20782.0,13246.0,1.82,26.52,13.36,43.43,39.0,1.57,0.81,32.61,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.84,H-LC,86.17,100811.0,17008.0,6986.0,2.32,20.29,6.93,28.63,159.0,2.43,0.82,34.71,AR,ATH,INSURANCE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.00,-842.94,H-SC,100.00,70746.0,-24570.0,69267.0,-0.81,-25.78,97.91,46.89,156.0,-0.35,0.57,39.54,X40N,NTT,MISC
55,PFIZER,4245.50,6318.30,2.04,H-SC,98.94,117942.0,20296.0,27374.0,-0.22,20.78,23.21,48.82,185.0,0.74,0.96,31.02,X40,ATH,PHARMA
78,TANLA,917.30,1963.11,-39.41,H-SC,97.87,148874.0,-90541.0,363491.0,1.47,-37.82,244.16,114.01,17.0,-0.25,1.21,36.54,AR,ATH,IT
20,COFFEEDAY,59.14,80.00,-55.73,L-SC,96.81,64800.0,-48749.0,88802.0,-3.05,-42.93,137.04,35.27,182.0,-0.55,0.53,58.60,XR,NTT,HOTELS
92,WHIRLPOOL,1167.49,2270.00,-36.30,M-SC,95.74,185676.0,14055.0,148021.0,-0.32,8.19,79.72,94.43,24.0,0.09,1.51,37.04,X40,NTT,DURABLES


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.55
1,25,39.17
2,50,68.94


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    33.33
X40N    14.11
XY25    13.95
X40     13.84
AR      12.23
XR      10.16
X200     1.23
SR       1.16
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.91
M-SC    18.70
X-LC    16.11
H-MC     8.63
M-LC     7.99
H-LC     6.71
X-MC     5.83
M-MC     5.75
X-SC     2.89
L-SC     2.42
L-LC     1.21
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,12.44,-1.53,34.27
IT,9.53,-19.81,86.23
BANKS,9.52,-12.85,65.60
MISC,6.73,-30.38,88.05
FINANCE,6.24,-24.60,83.76
INSURANCE,6.21,1.74,27.56
PAINTS,5.65,-18.31,53.33
AUTO,4.64,-9.52,57.36
ELECTRICAL,4.46,-5.49,59.17


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2871997.0,26
AR,1212498.0,13
X40N,1210041.0,17
XR,1098076.0,14
X40,724531.0,11
XY25,476963.0,10
SR,202038.0,2
X200,64229.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2320805.0,21
M-SC,2013067.0,21
X-LC,710272.0,12
M-MC,559043.0,5
H-MC,462915.0,8
X-SC,352395.0,4
M-LC,322537.0,6
X-MC,318571.0,5
L-SC,300269.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      918853.0      6
M-SC       XY24      746557.0      6
H-SC       AR        557890.0      3
           X40N      508773.0      6
M-SC       X40N      457569.0      6
M-MC       XY24      394059.0      3
X-LC       X40       392890.0      5
M-SC       XR        295669.0      4
H-SC       XR        281202.0      4
M-SC       X40       216094.0      2
H-LC       AR        214330.0      4
M-LC       XY24      196607.0      4
H-MC       XY24      182062.0      2
L-SC       XR        153002.0      2
X-LC       AR        140436.0      2
X-MC       XY24      139719.0      2
X-SC       XR        120909.0      1
M-SC       XY25      119421.0      1
X-SC       SR        117497.0      1
           X40N      113989.0      2
X-LC       XY25      112717.0      4
M-LC       XR        106855.0      1
X-MC       X40N      101618.0      2
M-SC       AR         93216.0      1
           SR         84541.0      1
L-SC       XY24       83077.0      1
M-MC       XR         83012.0      1
           AR         81972.0      1
X-MC       XY25       77234.0      1
H-MC       XY25       73410.0      1
X-LC       X200       64229.0      1
L-SC       AR         64190.0      1
H-MC       X40        61460.0      2
           AR         60464.0      1
           XR         57427.0      1
H-SC       X40        54087.0      2
L-LC       XY25       41045.0      1
H-LC       XY25       34061.0      1
H-MC       X40N       28092.0      1
M-LC       XY25       19075.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
